In [1]:
import boto3
import pandas as pd
from io import StringIO  # python3; python2: BytesIO
import time

In [2]:
AWS_ACCESS_KEY = "----"
AWS_SECRET_KEY = "----"
AWS_REGION = "us-east-1"
SCHEMA_NAME = "datawarehouse"
S3_STAGING_DIR = "s3://test-etl-datawarehouse/output/"
S3_BUCKET_NAME = "test-etl-datawarehouse"
S3_OUTPUT_DIRECTORY = "output"

In [3]:
athena_client = boto3.client(
    "athena",
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
    region_name=AWS_REGION,
)


In [ ]:
athena_client

In [ ]:
Dict = {}

def download_and_load_query_results(
    client: boto3.client, query_response: Dict
) -> pd.DataFrame:
    while True:
        try:
            client.get_query_results(
                QueryExecutionId=query_response["QueryExecutionId"]
            )
            break
        except Exception as err:
            if "not yet finished" in str(err):
                time.sleep(0.001)
            else:
                raise err
    temp_file_location: str = "athena_query_results.csv"
    s3_client = boto3.client(
        "s3",
        aws_access_key_id=AWS_ACCESS_KEY,
        aws_secret_access_key=AWS_SECRET_KEY,
        region_name=AWS_REGION,
    )
    s3_client.download_file(
        S3_BUCKET_NAME,
        f"{S3_OUTPUT_DIRECTORY}/{query_response['QueryExecutionId']}.csv",
        temp_file_location,
    )
    return pd.read_csv(temp_file_location)


In [ ]:
response = athena_client.start_query_execution(
QueryString="SELECT * FROM enigma_jhud",
QueryExecutionContext={"Database": SCHEMA_NAME},
ResultConfiguration={ "OutputLocation": S3_STAGING_DIR,
"EncryptionConfiguration": {"EncryptionOption": "SSE_S3"}, },
)

enigma_jhud = download_and_load_query_results(athena_client, response)

enigma_jhud.head()
# 11

,fips,admin2,province_state,country_region,last_update,latitude,longitude,confirmed,deaths,recovered,active,combined_key
0,NaN,NaN,Anhui,China,2020-01-22T17:00:00,31.826,117.226,1.0,NaN,NaN,NaN,"""Anhui"
1,NaN,NaN,Beijing,China,2020-01-22T17:00:00,40.182,116.414,14.0,NaN,NaN,NaN,"""Beijing"
2,NaN,NaN,Chongqing,China,2020-01-22T17:00:00,30.057,107.874,6.0,NaN,NaN,NaN,"""Chongqing"
3,NaN,NaN,Fujian,China,2020-01-22T17:00:00,26.079,117.987,1.0,NaN,NaN,NaN,"""Fujian"
4,NaN,NaN,Gansu,China,2020-01-22T17:00:00,36.061,103.834,NaN,NaN,NaN,NaN,"""Gansu"


In [ ]:
enigma_jhud.shape

(10002, 12)

In [ ]:
# fips		province_state	country_region	last_update	latitude	longitude	confirmed	deaths	recovered	active	combined_key
factCovid_1= enigma_jhud[['fips', 'province_state', 'country_region', 'confirmed', 'deaths', 'recovered', 'active']]
factCovid_2= states_daily[['fips', 'date', 'positive', 'negative', 'hospitalizedcurrently','hospitalized','hospitalizeddischarged']]
factCovid = pd.merge(factCovid_1, factCovid_2, on='fips', how='inner')



In [ ]:
factCovid.head()

,fips,province_state,country_region,confirmed,deaths,recovered,active,date,positive,negative,hospitalizedcurrently,hospitalized,hospitalizeddischarged
0,72.0,Puerto Rico,US,3.0,0.0,0.0,NaN,20210307,101327,305972.0,147.0,NaN,NaN
1,72.0,Puerto Rico,US,3.0,0.0,0.0,NaN,20210306,101327,305972.0,147.0,NaN,NaN
2,72.0,Puerto Rico,US,3.0,0.0,0.0,NaN,20210305,101066,305972.0,136.0,NaN,NaN
3,72.0,Puerto Rico,US,3.0,0.0,0.0,NaN,20210304,100867,305972.0,171.0,NaN,NaN
4,72.0,Puerto Rico,US,3.0,0.0,0.0,NaN,20210303,100765,305972.0,169.0,NaN,NaN


In [ ]:
dimRegion_1 = enigma_jhud [['fips', 'province_state', 'country_region', 'latitude', 'longitude']]
dimRegion_2 = nytimes_data_in_usa_us_county [['fips', 'county', 'state']]
dimRegion = pd.merge(dimRegion_1, dimRegion_2, on='fips', how='inner')




In [ ]:
dimHospital = rearc_usa_hospital_beds[['fips', 'state_name', 'latitude', 'longtitude', 'hq_address', 'hospital_name', 'hospital_type', 'hq_city', 'hq_state']]


In [ ]:
dimDate = states_daily[['fips', 'date']]
dimDate.head()


,fips,date
0,2,20210307
1,1,20210307
2,5,20210307
3,60,20210307
4,4,20210307


In [ ]:
dimDate['date'] = pd.to_datetime(dimDate['date'], format='%Y%m%d')


In [ ]:
dimDate.loc[:, 'date'] = pd.to_datetime(dimDate['date'], format='%Y%m%d')


In [ ]:
dimDate['month'] = dimDate['date'].dt.month
dimDate['year'] = dimDate['date'].dt.year
dimDate['day_of_week'] = dimDate['date'].dt.dayofweek


In [ ]:
# Convert to datetime format
dimDate['date'] = pd.to_datetime(dimDate['date'], format='%m/%d/%Y')

# Format as YYYY-MM-DD
dimDate['date'] =dimDate['date'].dt.strftime('%Y-%m-%d')
dimDate.head()

In [ ]:
dimDate.head()

In [ ]:
output_bucket = '' # already created on S3

csv_buffer = StringIO()
csv_buffer

In [ ]:
csv_buffer = StringIO()
dimDate.to_csv(csv_buffer,index=False)

s3_resource =boto3.resource('s3')
s3_resource.Object(output_bucket, 'dimDate.csv').put(Body=csv_buffer.getvalue())


In [ ]:
csv_buffer = StringIO()
dimRegion.to_csv(csv_buffer,index=False)

s3_resource =boto3.resource('s3')
s3_resource.Object(output_bucket, 'dimRegion.csv').put(Body=csv_buffer.getvalue())


In [ ]:
csv_buffer = StringIO()
dimHospital.to_csv(csv_buffer,index=False)

s3_resource =boto3.resource('s3')
s3_resource.Object(output_bucket, 'dimHospital.csv').put(Body=csv_buffer.getvalue())


In [ ]:
csv_buffer = StringIO()
factCovid.to_csv(csv_buffer,index=False)

s3_resource =boto3.resource('s3')
s3_resource.Object(output_bucket, 'factCovid.csv').put(Body=csv_buffer.getvalue())

In [ ]:
import pandas as pd

# Assuming dimDate, factCovid, dimRegion, and dimHospital are predefined DataFrames

dimDatesql = pd.io.sql.get_schema(dimDate.reset_index(), 'dimDate')
print(''.join(dimDatesql))

factCovidsql = pd.io.sql.get_schema(factCovid.reset_index(), 'factCovid')
print(''.join(factCovidsql))

dimRegionsql = pd.io.sql.get_schema(dimRegion.reset_index(), 'dimRegion')
print(''.join(dimRegionsql))

dimHospitalsql = pd.io.sql.get_schema(dimHospital.reset_index(), 'dimHospital')
print(''.join(dimHospitalsql))


In [ ]:
pip install redshift-connector

In [ ]:
import redshift_connector
import logging

# logging.basicConfig(level=logging.DEBUG)

try:
    conn = redshift_connector.connect(
        host='redshift-cluster-2.czfgt1tbsuck.us-east-2.redshift.amazonaws.com',
        database='dev',
        user='---',
        password='-----',
        ssl=True,
        timeout=60  # Increase the timeout if the connection takes longer
    )
    print(conn)
except redshift_connector.Error as e:
    print(f"Error: {e}")


In [ ]:
# Enabling autocommit
conn.autocommit = True

# Creating a cursor object to interact with the database
cursor = conn.cursor()

# Creating the dimDate table
cursor.execute("""
CREATE TABLE "dimDate" (
    "fips" INTEGER,
    "date" TIMESTAMP,
    "year" INTEGER,
    "month" INTEGER,
    "day_of_week" INTEGER
)
""")


cursor.execute("""
CREATE TABLE "dimHospital" (
  "fips" REAL,
  "state_name" TEXT,
  "latitude" REAL,
  "longitude" REAL,
  "hq_address" TEXT,
  "hospital_name" TEXT,
  "hospital_type" TEXT,
  "hq_city" TEXT,
  "hq_state" TEXT
)
""")



In [ ]:
# Enabling autocommit
conn.autocommit = True

# Creating a cursor object to interact with the database
cursor = conn.cursor()

# Creating the dimDate table
cursor.execute("""
CREATE TABLE "factCovid" (
  "fips" REAL,
  "province_state" TEXT,
  "country_region" TEXT,
  "confirmed" REAL,
  "deaths" REAL,
  "recovered" REAL,
  "active" REAL,
  "date" INTEGER,
  "positive" INTEGER,
  "negative" REAL,
  "hospitalized_currently" REAL,
  "hospitalized" REAL,
  "hospitalized_discharged" REAL
)
""")

# Creating the dimRegion table
cursor.execute("""
CREATE TABLE "dimRegion" (
  "fips" REAL,
  "province_state" TEXT,
  "country_region" TEXT,
  "latitude" REAL,
  "longitude" REAL,
  "county" TEXT,
  "state" TEXT
)
""")



In [ ]:
cursor.execute("""
copy factcovid from 's3://aws-session-datawarehousing-output/factCovid.csv'
credentials 'aws_iam_role=arn:aws:iam::975050334693:role/service-role/AmazonRedshift-CommandsAccessRole-20240625T125922'
delimiter ','
region 'us-east-2'
IGNOREHEADER 1;
""")

cursor.execute("""
copy dimregion from 's3://aws-session-datawarehousing-output/dimRegion.csv'
credentials 'aws_iam_role=arn:aws:iam::975050334693:role/service-role/AmazonRedshift-CommandsAccessRole-20240625T125922'
delimiter ','
region 'us-east-2'
IGNOREHEADER 1;
""")


cursor.execute("""
copy dimhospital from 's3://aws-session-datawarehousing-output/dimHospital.csv'
credentials 'aws_iam_role=arn:aws:iam::975050334693:role/service-role/AmazonRedshift-CommandsAccessRole-20240625T125922'
delimiter ','
region 'us-east-2'
IGNOREHEADER 1;
""")


cursor.execute("""
copy dimdate from 's3://aws-session-datawarehousing-output/dimDate.csv'
credentials 'aws_iam_role=arn:aws:iam::975050334693:role/service-role/AmazonRedshift-CommandsAccessRole-20240625T125922'
delimiter ','
region 'us-east-2'
IGNOREHEADER 1;
""")